In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, lower, regexp_replace, split, to_date, translate

spark= SparkSession.builder.config('spark.ui.port','4040').getOrCreate()

In [71]:
paper = spark.read.csv("file:///N/project/mag/mag-2021-01-05/mag/Papers.txt",\
                         header = False,\
                         sep=r'\t')
#paper.show(5)

In [72]:
def clean_text(c):
  c = regexp_replace(c, '"' , '')
  c = regexp_replace(c, '\\\\' , '')
  #c = regexp_replace(c, "(https?\://)\S+", "")
  c = regexp_replace(c, '$s/"//g', '')
  #c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  #c = split(c, "\\s+") tokenization...
  #c = regexp_replace(c, "([\d\.]+)", '')
  #c = replace(c, "\201C", '')
  return c

In [73]:
paper = paper.select(
                     paper._c0.cast("string").alias("paperId"),
                     paper._c1.cast("int").alias("rank"),
                     paper._c2.cast("string").alias("doi"),
                     paper._c3.cast("string").alias("docType"),
                     paper._c4.cast("string").alias("paperTitle"),
                     clean_text(col("_c5")).alias("originalTitle"),
                     clean_text(col("_c6")).alias("bookTitle"),
                     paper._c7.cast("int").alias("year"),
                     to_date(paper._c8, "yyyy-MM-dd").alias("Date"),
                     paper._c9.cast("string").alias("onlineDate"),
                     clean_text(col("_c10")).alias("publisher"),
                     clean_text(col("_c11")).alias("journalId"),
                     paper._c12.cast("string").alias("conferenceSeriesId"),
                     paper._c13.cast("string").alias("conferenceInstanceId"),
                     paper._c14.cast("string").alias("volume"),
                     paper._c15.cast("string").alias("issue"),
                     paper._c16.cast("string").alias("firstPage"),
                     paper._c17.cast("string").alias("lastPage"),
                     paper._c18.cast("int").alias("referenceCount"),
                     paper._c19.cast("string").alias("citationCount"),
                     paper._c20.cast("string").alias("estimatedCitation"),
                     clean_text(col("_c21")).alias("originalVenue"),
                     paper._c22.cast("string").alias("familyId"),
                     paper._c23.cast("string").alias("familyRank"),
                     to_date(paper._c24, "yyyy-MM-dd").alias("createdDate")  
)   

In [74]:
%%time

paper2 = paper.filter(paper.paperId.contains('2758718266'))

paper2.show()

+----------+-----+----+-------+--------------------+--------------------+---------+----+----+----------+---------+---------+------------------+--------------------+------+-----+---------+--------+--------------+-------------+-----------------+-------------+--------+----------+-----------+
|   paperId| rank| doi|docType|          paperTitle|       originalTitle|bookTitle|year|Date|onlineDate|publisher|journalId|conferenceSeriesId|conferenceInstanceId|volume|issue|firstPage|lastPage|referenceCount|citationCount|estimatedCitation|originalVenue|familyId|familyRank|createdDate|
+----------+-----+----+-------+--------------------+--------------------+---------+----+----+----------+---------+---------+------------------+--------------------+------+-----+---------+--------+--------------+-------------+-----------------+-------------+--------+----------+-----------+
|2758718266|28215|null|   null|ossidazione a 900...|Ossidazione a 900...|     null|null|null|      null|     null|     null|      

In [75]:
%%time

paper.coalesce(64).write.option("header","true") \
                               .option("sep","\t") \
                               .option("quoteAll", False) \
                               .option("emptyValue", None) \
                               .option("nullValue", None) \
                               .mode("overwrite") \
                               .csv('/N/project/mag/mag_jg_2021_update/nodes/papers')               

CPU times: user 72.7 ms, sys: 44.5 ms, total: 117 ms
Wall time: 7min 23s
